In [ ]:
import pandas as pd
import numpy as np
from alpha_vantage.cryptocurrencies import CryptoCurrencies
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

AlphaVantage_api_key = "INSERT-YOUR-API-KEY-HERE"
cc = CryptoCurrencies(key=AlphaVantage_api_key, output_format='pandas')

# get data
df, meta_data = cc.get_digital_currency_daily(symbol='BTC', market='USD')

# prep column names, index etc.
cols = ['1a. open (USD)', '2a. high (USD)', '3a. low (USD)', '4a. close (USD)', '5. volume']
df.index = pd.to_datetime(df.index)
df = df[cols].sort_index()
df.columns = ['open', 'high', 'low', 'close', 'vol']

display(meta_data)

In [ ]:
df.describe().transpose()

In [ ]:
df[['close', 'vol']].plot(subplots=True, layout=(2,1))

In [ ]:
# (1) dane stacjonarne
df_pct = df.pct_change()[1:]

# (2) Y
y = df['close'].pct_change().shift(periods=-1).fillna(0)[1:]

# (3) Podzial TRAIN/TEST
test_beg = '2020-12-30'

# podzial train/test
train_mask = df_pct.index<test_beg
test_mask = df_pct.index>=test_beg

cols = list(df.columns)
#cols = ['open', 'high', 'low', 'close', 'vol']

x_train = df_pct[cols][train_mask].astype(float)
x_test = df_pct[cols][test_mask].astype(float)

y_train = y[train_mask].astype(float)
y_test = y[test_mask].astype(float)

assert(len(x_train)==len(y_train))
assert(len(x_test)==len(y_test))

# (4) Normalizacja
scaler_df = StandardScaler() 

#budujuemy scaler na danych TRAIN only
scaler_df.fit_transform(x_train)

#aplikujemy skaler na dane
x_train = scaler_df.transform(x_train)
x_test = scaler_df.transform(x_test)

# (5) Mieszanie próbek TRAIN
x_train, y_train = shuffle(x_train, y_train, random_state=0)

In [ ]:
# (6) Uczenie sieci na danych TRAIN
def my_model(feat):
    
    x_input = Input(shape = (feat,))
    x = Dense(20, name='hidden')(x_input)
    x_output = Dense(1, activation='linear', name='out')(x)

    model = Model(inputs = x_input, outputs = x_output, name='way2becomefcknrich_btc')
    
    return model

#num features
feat = np.shape(x_train)[1]

model = my_model(feat)
model.compile(loss='mean_squared_error')
model.summary()

model.fit(x_train, y_train, epochs=100)

In [ ]:
# (7) Predykcja dla TEST
y_predicted = model.predict(x_test)

df_pred = pd.concat([y_test, pd.DataFrame(y_predicted, index=y_test.index, columns=['prediction'])], axis=1)
df_pred[df_pred.index>'2021-01-01'].plot(grid=True)

# (8) Wypłata zarobionej gotówki
mask = np.sign(df_pred['close']) == np.sign(df_pred['prediction'])

acc = np.sum(mask) / len(df_pred)
trans_positive = df_pred['close'][mask].abs()
trans_loss = df_pred['close'][~mask].abs()

print('accuracy:%.2f%% trans_positive:%.2f%% trans_loss:%.2f%%' %(100*acc, 100*trans_positive.sum(), 100*trans_loss.sum()))
print('zarobek netto:%.2f%%\n\n' %(100*trans_positive.sum() - 100*trans_loss.sum()))

Dodanie danych ACB / Aurora Cannabis

In [ ]:
from alpha_vantage.timeseries import TimeSeries

# get data
ts = TimeSeries(key=AlphaVantage_api_key, output_format='pandas')
df_acb, meta_data = ts.get_daily(symbol='ACB', outputsize='full')

# prep column names, index etc.
df_acb.columns = ['open_acb', 'high_acb', 'low_acb', 'close_acb', 'vol_acb']
df_acb.index = pd.to_datetime(df_acb.index)
df_acb.sort_index(inplace=True)

In [ ]:
# concatenate
df_new = pd.concat([df, df_acb], axis=1)
df_new.dropna(subset=['open'], inplace=True)
df_new.fillna(method='ffill', inplace=True)

In [ ]:
df_new.describe().transpose()

In [ ]:
import matplotlib.pyplot as plt
plt.hist2d(df_new['close'], df_new['close_acb'], bins=(20, 20))
plt.colorbar()
plt.xlabel('BTCUSD')
plt.ylabel('Cannabis')

In [ ]:
# (1) dane stacjonarne
df_newpct = df_new.pct_change()[1:]

# (2) Y
y = df_new['close'].pct_change().shift(periods=-1).fillna(0)[1:]

# (3) Podzial TRAIN/TEST
test_beg = '2020-12-30'

# podzial train/test
train_mask = df_newpct.index<test_beg
test_mask = df_newpct.index>=test_beg

cols = list(df_new.columns)
#cols = ['open', 'high', 'low', 'close', 'vol']

x_train = df_newpct[cols][train_mask].astype(float)
x_test = df_newpct[cols][test_mask].astype(float)

y_train = y[train_mask].astype(float)
y_test = y[test_mask].astype(float)

assert(len(x_train)==len(y_train))
assert(len(x_test)==len(y_test))

# (4) Normalizacja
scaler_df = StandardScaler() 

#budujuemy scaler na danych TRAIN only
scaler_df.fit_transform(x_train)

#aplikujemy skaler na dane
x_train = scaler_df.transform(x_train)
x_test = scaler_df.transform(x_test)

# (5) Mieszanie próbek TRAIN
x_train, y_train = shuffle(x_train, y_train, random_state=0)

In [ ]:
df_newpct.describe().transpose()

In [ ]:
df_newpct[['open_acb','high_acb','low_acb','close_acb']][df_newpct.index=='2020-05-11']

In [ ]:
#korekcja reverse split 1:12
for col in ['open_acb','high_acb','low_acb','close_acb']:
    df_newpct.loc[df_newpct.index=='2020-05-11', col] = (df_newpct.loc[df_newpct.index=='2020-05-11', col] -11 ) / 12
df_newpct[['open_acb','high_acb','low_acb','close_acb']][df_newpct.index=='2020-05-11']

In [ ]:
plt.hist2d(x_train[:,3], x_train[:,8], bins=(20, 20), range=[[-1,1],[-0.2,0.2]])
plt.colorbar()
plt.xlabel('BTCUSD')
plt.ylabel('Cannabis')

In [ ]:
#num features
feat = np.shape(x_train)[1]

model = my_model(feat)
model.compile(loss='mean_squared_error')
model.summary()

model.fit(x_train, y_train, epochs=20)

In [ ]:
# (7) Predykcja dla TEST
y_predicted = model.predict(x_test)

df_pred = pd.concat([y_test, pd.DataFrame(y_predicted, index=y_test.index, columns=['prediction'])], axis=1)
df_pred[df_pred.index>'2021-01-01'].plot(grid=True)

# (8) Wypłata zarobionej gotówki
mask = np.sign(df_pred['close']) == np.sign(df_pred['prediction'])

acc = np.sum(mask) / len(df_pred)
trans_positive = df_pred['close'][mask].abs()
trans_loss = df_pred['close'][~mask].abs()

print('accuracy:%.2f%% trans_positive:%.2f%% trans_loss:%.2f%%' %(100*acc, 100*trans_positive.sum(), 100*trans_loss.sum()))
print('zarobek netto:%.2f%%\n\n' %(100*trans_positive.sum() - 100*trans_loss.sum()))